In [1]:
import numpy as np
import h5py

from keras.models import Sequential
from keras.layers import Embedding, Dropout, Dense, Activation
from keras.layers import LSTM, Bidirectional, Merge, RepeatVector

Using TensorFlow backend.


In [2]:
with h5py.File('context.h5', 'r') as hf:
    context_array = hf['context'][:]
with h5py.File('questions.h5', 'r') as hf:
    question_array = hf['questions'][:]
with h5py.File('answers.h5', 'r') as hf:
    answer_array = hf['answers'][:]
with h5py.File('begin.h5', 'r') as hf:
    begin_span = hf['begin'][:]
with h5py.File('end.h5', 'r') as hf:
    end_span = hf['end'][:]

In [3]:
print context_array.shape
print question_array.shape
print answer_array.shape
print begin_span.shape
print end_span.shape

(87599, 1000)
(87599, 100)
(87599, 50)
(87599,)
(87599,)


In [4]:
vocab_size = 37
embedding_vector_length = 128
max_span = 1000
batch = 512

In [5]:
context_model = Sequential()
context_model.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_vector_length, input_length=1000))
context_model.add(Bidirectional(LSTM(256, implementation=2), merge_mode='mul'))
context_model.add(Dropout(0.5))
context_model.add(RepeatVector(10))

In [6]:
ques_model = Sequential()
ques_model.add(Embedding(input_dim=vocab_size,
                        output_dim=embedding_vector_length, input_length=100))
ques_model.add(Bidirectional(LSTM(256, implementation=2), merge_mode='mul'))
ques_model.add(Dropout(0.5))
ques_model.add(RepeatVector(10))

In [ ]:
# Merge layers
model = Sequential()
model.add(Merge([context_model, ques_model], mode='mul'))
model.add(Bidirectional(LSTM(256, implementation=2), merge_mode='mul'))
model.add(Dropout(0.5))
model.add(Dense(max_span))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', 
             metrics=['accuracy'])
model.summary()

model_history = model.fit([context_array, question_array], begin_span,
                         verbose=2, batch_size=batch, epochs=10)